In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Load Data

In [ ]:
#images are in ./dataset/khalil and ./dataset/others
# labels are the folder names
# load the data

import os
import numpy as np

data_path = './dataset'
folders = os.listdir(data_path)
labels = []
images = []

# each image has a label. labels are the folder names.
for folder in folders:
    folder_path = os.path.join(data_path, folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        images.append(image_path)
        labels.append(folder)

# convert to numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images)
print(labels)


# Image Preprocessing

In [ ]:
 #detect faces in the image and extract the face

import cv2
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face


In [ ]:
# function to extract faces from an image
def faces_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    faces_list = []
    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        faces_list.append(cropped_face)
    return faces_list


testing the face detection

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for image in images:
    img = mpimg.imread(image)

    face = face_extractor(img)
    if face is not None:
        plt.imshow(face)
        plt.show()
    else:
        print('Face not found')

Save the faces in "processed" folder

In [ ]:
# save the faces in "processed" folder. The folder will be created if it doesn't exist.
# the faces are saved with the name of the folder they belong to (label) and a number.
processed_path = './processed'
if not os.path.exists(processed_path):
    os.mkdir(processed_path)
#clean the folder
for folder in os.listdir(processed_path):
    folder_path = os.path.join(processed_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        os.remove(file_path)
    os.rmdir(folder_path)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for i, image in enumerate(images):
    img = cv2.imread(image)
    face = face_extractor(img)
    if face is not None:
        face = cv2.resize(face, (256, 256))
    
        save_path = os.path.join(processed_path, labels[i])
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        file_name_path = os.path.join(save_path, labels[i] + str(i) + '.jpg')
        cv2.imwrite(file_name_path, face)
        cv2.waitKey(0)
    else:
        print('Face not found')
        pass



# CNN Model


## Load data and divide into train and test sets

In [ ]:
data = './processed/'
batch_size = 32
image_size=(256,256)
training_set = tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=0.5,
    subset="training",
    image_size=image_size,
    seed=123,
    batch_size=batch_size,
)

testing_set = tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=0.5,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,)

In [ ]:
class_names = training_set.class_names
print(class_names)

In [ ]:
# Visualizing the data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in training_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", input_shape=(256, 256, 3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),]
)

In [ ]:
# generating train and test data
number_of_augments=3
_aug_ds_new = training_set.unbatch()
aug_ds_new = training_set.shuffle(100).map(lambda x, y: (data_augmentation(x), y))

aug_ds_new = aug_ds_new.unbatch()
aug_ds_new = aug_ds_new.concatenate(_aug_ds_new)


In [ ]:
for i in range(number_of_augments):
    aug_ds_=training_set.shuffle(100).map(lambda x, y: (data_augmentation(x), y))
    print(i)
    for images, labels in aug_ds_:
        print(images.shape)
    
    print('-----------------')
    aug_ds_new=aug_ds_new.concatenate(aug_ds_.unbatch())
    for images, labels in aug_ds_new:
        print(images.shape)
    print('-----------------')

In [ ]:
aug_ds_new=aug_ds_new.batch(32)
train_data_augmented = aug_ds_new

In [ ]:
for image_batch, labels_batch in aug_ds_new:
    print(image_batch.shape)
    print(labels_batch.numpy())

A basic Keras model

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(256, 256, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Compile the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
#Train the model (Train the model for 10 epochs with the Keras Model.fit method):

#callback = tf.keras.callbacks.EarlyStopping(monitor='loss',  patience=3)
# This callback will stop the training when there is no improvement in the loss for 2 consecutive epochs.

#history = model.fit(
  #training_set,
  #validation_data=testing_set,
  #epochs=20, 
  #callbacks=[callback],)
#epochs=len(history.history['loss'])

Train the model 

In [ ]:
epochs = 20
history = model.fit(
    train_data_augmented,
    validation_data=testing_set,
    epochs=epochs
)

Visualize training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Predict on new data


In [ ]:
im_path = 'malek9.jpg'

img = tf.keras.utils.load_img(
    im_path, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)